In [42]:
#Dependencies 
import matplotlib
import pandas as pd
import json
import numpy as np
import requests

In [43]:
#import csv
ontimeMarketing = "T_ONTIME_MARKETING.csv"
airlineData = pd.read_csv(ontimeMarketing, encoding="utf-8",dtype=object)
cleanAirlineData = airlineData[["ORIGIN_CITY_NAME", 
                                "ORIGIN_STATE_ABR", 
                                "DEST_CITY_NAME", 
                                "DEST_STATE_ABR", 
                                "CRS_DEP_TIME", 
                                "DEP_TIME", 
                                "DEP_DELAY", 
                                "TAXI_OUT", 
                                "TAXI_IN", 
                                "ARR_TIME", 
                                "ARR_DELAY", 
                                "CANCELLED", 
                                "CANCELLATION_CODE", 
                                "DIVERTED", 
                                "CARRIER_DELAY", 
                                "WEATHER_DELAY", 
                                "NAS_DELAY", 
                                "SECURITY_DELAY", 
                                "LATE_AIRCRAFT_DELAY", 
                                "DIV_AIRPORT_LANDINGS", 
                                "DIV_ACTUAL_ELAPSED_TIME"]].fillna(0.00)
cleanAirlineData

,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_TIME,...,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DIV_ACTUAL_ELAPSED_TIME
0,"New York, NY",NY,"Los Angeles, CA",CA,0900,0852,-8,18,5,1124,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,"Los Angeles, CA",CA,"New York, NY",NY,1235,1232,-3,17,12,2107,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,"Boston, MA",MA,"Los Angeles, CA",CA,1800,1754,-6,18,8,2041,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,"Los Angeles, CA",CA,"Boston, MA",MA,0750,0742,-8,20,15,1638,...,0,0.0,0,0,0,19,0,0,0,0.0
4,"Phoenix, AZ",AZ,"Portland, OR",OR,1502,1458,-4,22,5,1654,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582420,"Phoenix, AZ",AZ,"Los Angeles, CA",CA,1200,1203,3,12,5,1218,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
582421,"Seattle, WA",WA,"Denver, CO",CO,1800,1842,42,15,5,2218,...,0,0.0,0,0,0,6,0,42,0,0.0
582422,"Orlando, FL",FL,"Austin, TX",TX,0705,0703,-2,18,4,0907,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
582423,"Los Angeles, CA",CA,"Sacramento, CA",CA,1315,1430,75,15,3,1549,...,0,0.0,0,69,0,0,0,0,0,0.0


In [44]:
#GEOAPIfy

In [45]:
#Filter to Chicago, IL flights only
chicagoAirlines = cleanAirlineData.loc[(cleanAirlineData["ORIGIN_CITY_NAME"] == "Chicago, IL") | 
(cleanAirlineData["DEST_CITY_NAME"] == "Chicago, IL")]

chicagoAirlines

,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_TIME,...,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DIV_ACTUAL_ELAPSED_TIME
121,"Chicago, IL",IL,"Washington, DC",VA,1613,1609,-4,14,3,1907,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
122,"Philadelphia, PA",PA,"Chicago, IL",IL,1340,1334,-6,13,9,1434,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
199,"Chicago, IL",IL,"Dallas/Fort Worth, TX",TX,0626,0627,1,11,14,0902,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
283,"Chicago, IL",IL,"Dallas/Fort Worth, TX",TX,0826,0822,-4,12,16,1040,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
333,"Phoenix, AZ",AZ,"Chicago, IL",IL,1956,2041,45,18,7,0057,...,0,0.0,0,29,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582373,"Chicago, IL",IL,"Phoenix, AZ",AZ,1315,1320,5,9,5,1544,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
582374,"Orlando, FL",FL,"Chicago, IL",IL,1045,1040,-5,21,5,1233,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
582388,"Chicago, IL",IL,"Denver, CO",CO,0730,0725,-5,13,6,0846,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
582396,"Dallas, TX",TX,"Chicago, IL",IL,0705,0700,-5,8,4,0903,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [46]:
#Comparison for reason of delay
#Counts of delay reasons
delayedFlights = chicagoAirlines.loc[(chicagoAirlines["DEP_DELAY"].astype(float)>0.0) | (chicagoAirlines["ARR_DELAY"].astype(float)> 0.0)]
delayedFlights [["ORIGIN_CITY_NAME", "ORIGIN_STATE_ABR", "DEST_CITY_NAME", "DEST_STATE_ABR", "DEP_DELAY", "TAXI_OUT", 
                 "TAXI_IN", "ARR_DELAY", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY",
                 "DIV_ACTUAL_ELAPSED_TIME"]]
delayedFlights

,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_TIME,...,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DIV_ACTUAL_ELAPSED_TIME
121,"Chicago, IL",IL,"Washington, DC",VA,1613,1609,-4,14,3,1907,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
199,"Chicago, IL",IL,"Dallas/Fort Worth, TX",TX,0626,0627,1,11,14,0902,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
333,"Phoenix, AZ",AZ,"Chicago, IL",IL,1956,2041,45,18,7,0057,...,0,0.0,0,29,0,0,0,0,0,0.0
386,"Chicago, IL",IL,"Phoenix, AZ",AZ,1855,1919,24,15,6,2222,...,0,0.0,0,7,0,1,0,17,0,0.0
387,"Phoenix, AZ",AZ,"Chicago, IL",IL,1321,1340,19,21,12,1817,...,0,0.0,0,19,0,6,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582158,"Boston, MA",MA,"Chicago, IL",IL,1230,1255,25,11,4,1413,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
582160,"Chicago, IL",IL,"Fort Lauderdale, FL",FL,1410,1411,1,6,7,1750,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
582361,"Chicago, IL",IL,"Washington, DC",VA,1000,0954,-6,14,9,1253,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
582368,"Nashville, TN",TN,"Chicago, IL",IL,0650,0650,0,38,7,0852,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [47]:
# Collect flight info for delay reason
delay_columns = ["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]
delay_data = delayedFlights[delay_columns]

# Convert delay column values to number
delay_data = delay_data.apply(pd.to_numeric, errors='coerce')

# Obtain counts for delay types based on delay type columns with a value greater than 0.0 for each row
delay_counts = {"CARRIER_DELAY": 0, "WEATHER_DELAY": 0, "NAS_DELAY": 0, "SECURITY_DELAY": 0, "LATE_AIRCRAFT_DELAY": 0}

for index, row in delay_data.iterrows():
    for col in delay_columns:
        if row[col] > 0.0:
            delay_counts[col] += 1

print("Counts for delay types")
for key, value in delay_counts.items():
    print(f"{key}: {value}")

Counts for delay types
CARRIER_DELAY: 7463
WEATHER_DELAY: 2276
NAS_DELAY: 8550
SECURITY_DELAY: 65
LATE_AIRCRAFT_DELAY: 7832


In [52]:
# Create a Bar Chart showing the different types of delays
delay_counts = {
    'Delay Type': ['Weather', 'Maintenance', 'Security', 'Weather', 'Security'],
    'Delay Count': [5, 3, 2, 7, 1]
}
delay_counts = pd.DataFrame(delay_counts)

# Filter the DataFrame to include only rows where the delay count is greater than 0
filtered_delay_data = delay_data[delay_data['Delay Count'] > 0.0]

# Extract delay types and their counts
delay_types = list(delay_counts.keys())
delay_counts_values = list(delay_counts.values())

#Plot bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(delay_types, delay_counts_values, color='blue')

# Add labels
plt.xlabel('Delay Type')
plt.ylabel('Delay Count')
plt.title('Counts of Different Types of Delays for Chicago, IL')
plt.xticks(rotation=45)
plt.show()

KeyError: 'Delay Count'